In [2]:
%load_ext autoreload
%autoreload

In [3]:
import sys
sys.path.insert(0, '../src')
from data_loader import GeneralDataLoader
from graph_data import StarDogGraph
from parent_ruleminer import RunParseAMIE
from tqdm import tqdm
from runner import run
import random

## args

In [5]:
f_name= 'DB15K_num'
base = '../data'
p = f'{base}/datasets/{f_name}'
PATH_RM = f"{base}/rule_miners/amie_jar/amie3.jar"
PATH_result = f"{base}/results/{f_name}/"

## dataloader

In [6]:
dl = GeneralDataLoader(path_t=f'{p}/train_dl.tsv', path_numerical_preds=f'{p}/numericals.tsv')
dl_test = GeneralDataLoader(path_t=f'{p}/test_dl.tsv', path_numerical_preds=f'{p}/numericals.tsv')

In [20]:
dl.df.head(3)

,subject,predicate,object
0,http://dbpedia.org/resource/Anarchism,http://www.w3.org/2000/01/rdf-schema%23seeAlso,http://dbpedia.org/resource/Franciaorsz%C3%A1g
1,http://dbpedia.org/resource/Alabama,http://dbpedia.org/ontology/country,http://dbpedia.org/resource/United_States
2,http://dbpedia.org/resource/Alabama,http://dbpedia.org/ontology/language,http://dbpedia.org/resource/English_Americans


## database

In [8]:
gr = StarDogGraph(dl, database_name='DB15K_num', force=True, p_save_g=PATH_result+'graph.ttl')

../data/results/DB15K_num/graph.ttl


48078it [00:02, 23916.95it/s]
72678it [00:02, 25485.98it/s]


saving...
../data/results/DB15K_num/graph.ttl


## Rule miner


In [9]:
amie = RunParseAMIE(data=dl.df, path_rule_miner=PATH_RM,
                    path_save_rules=PATH_result)
rules = amie.parse()

default is True java -jar ../data/rule_miners/amie_jar/amie3.jar ../data/results/DB15K_num/train.txt


Using the default schema relations


number of mined rules: 2689


In [21]:
print("some rules mined by AMIE",  *random.choices(rules, k=3), sep='\n')

some rules mined by AMIE
?f  http://dbpedia.org/ontology/executiveProducer  ?b  ?a  http://dbpedia.org/ontology/subsequentWork  ?f   => ?a  http://dbpedia.org/ontology/executiveProducer  ?b
?e  http://dbpedia.org/ontology/recordLabel  ?a  ?e  http://www.w3.org/1999/02/22-rdf-syntax-ns%23type  ?b   => ?a  http://www.w3.org/1999/02/22-rdf-syntax-ns%23type  ?b
?e  http://dbpedia.org/ontology/starring  ?a  ?e  http://www.w3.org/1999/02/22-rdf-syntax-ns%23type  ?b   => ?a  http://www.w3.org/1999/02/22-rdf-syntax-ns%23type  ?b


## Enrich Rules

In [18]:
dict_all_new_rules_f_score = run(rules[1324:1329], gr, list(dl.numerical_preds))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.02it/s]


In [19]:
dict_all_new_rules_f_score

{0: {'parent_rule': ?a  http://dbpedia.org/ontology/country  ?b  ?a  http://www.w3.org/2000/01/rdf-schema%23seeAlso  ?b   => ?a  http://dbpedia.org/ontology/region  ?b,
  'numerical_rules': [{'parent_rule': ?a  http://dbpedia.org/ontology/country  ?b  ?a  http://www.w3.org/2000/01/rdf-schema%23seeAlso  ?b   => ?a  http://dbpedia.org/ontology/region  ?b,
    'include_exclude': 'exclude',
    'numerical_part': {('?a',
      'http://dbpedia.org/ontology/populationTotal'): {'<=': 332052.0, '>': 179508.5}},
    'support': 8,
    'pca_body_size': 18,
    'pca_confidence': 0.4444444444444444,
    'head_coverage': 0.05333333333333334,
    'f_score': 0.09523809523809523,
    'level': 1},
   {'parent_rule': ?a  http://dbpedia.org/ontology/country  ?b  ?a  http://www.w3.org/2000/01/rdf-schema%23seeAlso  ?b   => ?a  http://dbpedia.org/ontology/region  ?b,
    'include_exclude': 'include',
    'numerical_part': {('?b',
      'http://www.w3.org/2003/01/geo/wgs84_pos%23long'): {'<=': 7.41708135604858